## OpenWeatherMap API

OpenWeatherMap es un servicio en línea que proporciona datos meteorológicos globales a través de una API, incluyendo datos meteorológicos actuales, pronósticos, pronósticos inmediatos y datos históricos para cualquier ubicación geográfica.

**_Documentación_**: https://openweathermap.org/api

### Current

Es un servicio que retorna datos meteorológicos actuales de un punto usando latitud y longitud.

**_Documentación_**: https://openweathermap.org/current
```html
endpoint: https://api.openweathermap.org/data/2.5/weather
```

`Params:`

- **lat, lon** : _required_ : Geographical coordinates (latitude, longitude). If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around, please use our Geocoding API.


- **appid** : _required_ : Your unique API key (you can always find it on your account page under the "API key" tab).


- **mode** : _optional_ : Response format. Possible values are xml and html. If you don't use the mode parameter format is JSON by default.


- **units** :  _optional_ : Units of measurement. standard, **metric** and imperial units are available. If you do not use the units parameter, standard units will be applied by default.


- **lang** : _optional_ : You can use this parameter to get the output in your language.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

import requests

from datetime import datetime
from time import sleep

from pprint import pprint

In [2]:
api_key = "cd31574dc65a379909d4f921033a7e96"

lat, lon = 40.416775, -3.703790 # Madrid, Spain

endpoint = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

response = requests.get(url = endpoint)

data = response.json()

pprint(data)

{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 40.4168, 'lon': -3.7038},
 'dt': 1698164280,
 'id': 3117735,
 'main': {'feels_like': 284.4,
          'humidity': 82,
          'pressure': 1014,
          'temp': 285.02,
          'temp_max': 286.29,
          'temp_min': 283.92},
 'name': 'Madrid',
 'rain': {'1h': 0.75},
 'sys': {'country': 'ES',
         'id': 2007545,
         'sunrise': 1698129290,
         'sunset': 1698168180,
         'type': 2},
 'timezone': 7200,
 'visibility': 7000,
 'weather': [{'description': 'moderate rain',
              'icon': '10d',
              'id': 501,
              'main': 'Rain'}],
 'wind': {'deg': 220, 'speed': 7.72}}


In [3]:
# Usando un diccionario como parámetros

params = {"appid" : api_key,
          "lat"   : lat,
          "lon"   : lon,
          "units" : "metric"}

endpoint = f"https://api.openweathermap.org/data/2.5/weather"

response = requests.get(url = endpoint, params = params)

data = response.json()

pprint(data, sort_dicts = False)

{'coord': {'lon': -3.7038, 'lat': 40.4168},
 'weather': [{'id': 501,
              'main': 'Rain',
              'description': 'moderate rain',
              'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 11.82,
          'feels_like': 11.22,
          'temp_min': 10.77,
          'temp_max': 13.14,
          'pressure': 1014,
          'humidity': 83},
 'visibility': 7000,
 'wind': {'speed': 7.72, 'deg': 220},
 'rain': {'1h': 0.61},
 'clouds': {'all': 75},
 'dt': 1698164515,
 'sys': {'type': 2,
         'id': 2007545,
         'country': 'ES',
         'sunrise': 1698129290,
         'sunset': 1698168180},
 'timezone': 7200,
 'id': 3117735,
 'name': 'Madrid',
 'cod': 200}


In [4]:
def get_current_weather(lat, lon, api_key):
    
    params = {"appid" : api_key,
              "lat"   : lat,
              "lon"   : lon,
              "units" : "metric"}

    endpoint = f"https://api.openweathermap.org/data/2.5/weather"

    response = requests.get(url = endpoint, params = params)

    return response.json()

In [5]:
# Lat & Lon de capitales de España

df = pd.read_html(io = "https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales")

df = df[1]

# Lat & Lon
df["Lat ETRS89"] = df["Lat ETRS89"]/100_000_000
df["Lon ETRS89"] = df["Lon ETRS89"]/100_000_000

# Renombrar columnas
df.rename(mapper  = {col : col.split(" ")[0] for col in df.columns if "ETRS89" in col},
          axis    = 1,
          inplace = True)

weather_data = list()

for lat, lon in df[["Lat", "Lon"]].values:

    data = get_current_weather(lat = lat, lon = lon, api_key = api_key)

    try:
        description = data["weather"][0]["description"]
        temp        = data["main"]["temp"]
        feels_like  = data["main"]["feels_like"]
        temp_min    = data["main"]["temp_min"]
        temp_max    = data["main"]["temp_max"]
        pressure    = data["main"]["pressure"]
        humidity    = data["main"]["humidity"]
        wind_speed  = data["wind"]["speed"]
        dt          = data["dt"]
        lat         = data["coord"]["lat"]
        lon         = data["coord"]["lon"]
        name        = data["name"]

        weather_data.append([name, description, temp, feels_like, temp_min, 
                             temp_max, pressure, humidity, wind_speed, dt, lat, lon])
    except:
        pass

    sleep(0.1)

df_weather = pd.DataFrame(data   = weather_data,
                         columns = ["capital", "description", "temp", "feels_like", "temp_min",
                                    "temp_max", "pressure", "humidity", "wind_speed", "dt", "lat", "lon"])

df_weather

,capital,description,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,dt,lat,lon
0,A Coruña,few clouds,19.48,19.55,18.63,20.24,1007,79,4.47,1698164557,43.3701,-8.3911
1,Albacete,clear sky,16.04,14.95,16.04,16.04,1016,48,7.20,1698164557,38.9959,-1.8557
2,Benidorm,clear sky,21.22,20.99,20.56,22.17,1014,61,7.33,1698164558,38.3455,-0.0483
3,Almería,few clouds,19.73,19.48,18.95,19.73,1014,66,9.77,1698164261,36.8371,-2.4606
4,Ávila,overcast clouds,9.01,6.85,9.01,9.01,1013,84,3.87,1698164471,40.6559,-4.6977
5,Badajoz,broken clouds,17.28,17.23,16.93,17.28,1014,83,5.14,1698164560,38.8787,-6.9710
6,"Sant Pere, Santa Caterina i La Ribera",few clouds,20.77,20.00,19.31,21.77,1010,42,8.75,1698164315,41.3829,2.1774
7,Bilbao,few clouds,19.36,18.58,16.72,20.12,1017,47,10.29,1698164465,43.2626,-2.9238
8,Burgos,mist,9.12,5.34,9.12,9.12,1011,93,8.75,1698164562,42.3411,-3.7042
9,Cáceres,light rain,13.27,12.95,13.27,13.27,1015,88,4.67,1698164563,39.4732,-6.3712


In [6]:
#pip install lxml
#me está dando error al leer el html, esta librería es para: 


### Air Pollution (Historical)

_**Documentación:**_ https://openweathermap.org/api/air-pollution

```html
endpoint: http://api.openweathermap.org/data/2.5/air_pollution/history
```

Air Pollution API proporciona datos actuales, pronósticos e históricos sobre la contaminación del aire para cualquier coordenada.

Además del Índice de Calidad del Aire (Air Quality Index - **aqi**) básico, la API devuelve datos sobre gases contaminantes, como:

- Carbono monóxido (**CO**)
- Nitrogeno monóxido (**NO**)
- Dióxido de nitrógeno (**NO2**)
- Ozono (**O3**)
- Dióxido de azufre (**SO2**)
- Amoniaco (**NH3**)
- Partículas (**PM2.5** y **PM10**)


| Qualitative name | Index | SO2 (μg/m3)       | NO2 (μg/m3)       | PM10 (μg/m3)      | PM2.5 (μg/m3)     | O3 (μg/m3)        | CO (μg/m3)           |
|------------------|-------|------------|------------|------------|------------|------------|---------------|
| Good             | 1     | [0; 20)    | [0; 40)    | [0; 20)    | [0; 10)    | [0; 60)    | [0; 4400)     |
| Fair             | 2     | [20; 80)   | [40; 70)   | [20; 50)   | [10; 25)   | [60; 100)  | [4400; 9400)  |
| Moderate         | 3     | [80; 250)  | [70; 150)  | [50; 100)  | [25; 50)   | [100; 140) | [9400-12400)  |
| Poor             | 4     | [250; 350) | [150; 200) | [100; 200) | [50; 75)   | [140; 180) | [12400; 15400)|
| Very Poor        | 5     | ⩾350       | ⩾200       | ⩾200      | ⩾75        | ⩾180       | ⩾15400       |


`Params:`
- **lat, lon** : _required_ :	Geographical coordinates (latitude, longitude). If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around, please use our Geocoding API.


- **appid** : _required_ : Your unique API key (you can always find it on your account page under the "API key" tab)


- **start** : _optional_ : Start date (unix time, UTC time zone), e.g. start=1369728000


- **end** : _optional_ : End date (unix time, UTC time zone), e.g. end=1369789200

In [ ]:
start = int(datetime(2023, 1, 1).timestamp()) # 2023-01-01

end = int(datetime.now().timestamp()) # Hoy

print(f"start: {start}\nend: {end}")

In [ ]:
params = {"appid" : api_key,
          "lat"   : lat,
          "lon"   : lon,
          "start" : start,
          "end"   : end}

endpoint = "http://api.openweathermap.org/data/2.5/air_pollution/history"

response = requests.get(url = endpoint, params = params)

data = response.json()

pprint(data)

In [ ]:
df_air = pd.json_normalize(data["list"])

df_air

In [ ]:
# Renombramos las columnas

df_air.rename(mapper  = {x : x.split(".")[1] for x in df_air.columns[1:]},
              axis    = 1,
              inplace = True)

In [ ]:
df_air

In [ ]:
# Timestamp a datetime

df_air["dt"] = df_air["dt"].apply(lambda x : datetime.fromtimestamp(x))

df_air

In [ ]:
px.line(data_frame = df_air,
        x          = "dt",
        y          = "no2")

In [ ]:
df_air["day_of_year"] = df_air["dt"].apply(lambda x : x.strftime("%Y-%m-%d"))

df_air.head()

In [ ]:
df_air2 = df_air.groupby("day_of_year", as_index = False).agg({col : "mean" for col in df_air.columns[1:-1]})

df_air2

In [ ]:
# Line plot: media de valores de componentes por día

fig = px.line(data_frame = df_air2,
              x          = "day_of_year",
              y          = df_air2.columns[1:])

fig.show()

In [ ]:
df_air["hour"] = df_air["dt"].apply(lambda x : x.hour)

df_air

In [ ]:
df_air3 = df_air.groupby("hour", as_index = False).agg({col : "mean" for col in df_air.columns[1:-2]})

df_air3

In [ ]:
# Area plot: media de componentes agrupados por hora
fig = px.area(data_frame = df_air3,
              x          = "hour",
              y          = df_air3.columns[1:])

fig.show()

In [ ]:
# Boxplot

fig = px.box(data_frame = df_air,
             x          = "hour",
             y          = df_air.columns[2],
             hover_data = ["day_of_year"])

fig.show()

In [ ]:
################################################################################################################################